In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import h5py
import copy

import amici
from petab.C import *
import petab
import petab.C
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.startpoint import uniform
from pypesto.engine import MultiProcessEngine, MultiThreadEngine
from pypesto.optimize.optimizer import FidesOptimizer
import pypesto.optimize as optimize

from pypesto.visualize import waterfall
from pypesto.visualize import parameters
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from pypesto.visualize import profiles

import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

In [2]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

def find_cut_off_x_trace(result, ci = 0.95, flatten = True):

    cut_off_value = scipy.stats.chi2.ppf(ci, 1)
    best_fval = result.optimize_result.list[0].fval

    # store the optimized x trace that are below the cut off value
    x_trace_within_cut_off = []
    if flatten:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off.append(x_trace[j])
    else:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            x_trace_within_cut_off_i = []
            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off_i.append(x_trace[j])
            x_trace_within_cut_off.append(x_trace_within_cut_off_i)

    return x_trace_within_cut_off

In [3]:
# Plot setting
plt.rcParams['font.size'] = 30

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

In [4]:
# number of optimization runs
n_runs = 5000

In [5]:
# Define the folder where you want to save the figures
folder_path = "/Users/yuhongliu/Documents/OV/figures/7_individual_start/n"+str(n_runs)+"/"

# If the folder does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# optimization
hierarchical = True

petab_yaml = 'petab_files_mean/dividing_infected_cells.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

np.random.seed(500)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"DIVIDING_INFECTED_CELLS_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

Compiling amici model to folder /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/DIVIDING_INFECTED_CELLS_Model.
2024-10-22 18:19:44.997 - amici.petab.sbml_import - INFO - Importing model ...
2024-10-22 18:19:44.997 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-10-22 18:19:45.007 - amici.petab.sbml_import - INFO - Model name is 'DIVIDING_INFECTED_CELLS_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/DIVIDING_INFECTED_CELLS_Model'.
2024-10-22 18:19:45.008 - amici.petab.sbml_import - INFO - Species: 0
2024-10-22 18:19:45.008 - amici.petab.sbml_import - INFO - Global parameters: 17
2024-10-22 18:19:45.008 - amici.petab.sbml_import - INFO - Reactions: 0
2024-10-22 18:19:45.018 - amici.petab.sbml_import - INFO - Observables: 1
2024-10-22 18:19:45.018 - amici.petab.sbml_import - INFO - Sigmas: 1
2024-10-22 18:19:45.019 - amici.petab.sbml_import - DEBUG - Adding output parameters to mod

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/DIVIDING_INFECTED_CELLS_Model -B /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/DIVIDING_INFECTED_CELLS_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/opt/homebrew/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/DIVIDING_INFECTED_CELLS_Model/DIVIDING_INFECTED_CELLS_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_MODULE_PATH=/opt/homebrew/lib/python3.12/site-packages/amici/lib/cmake/SuiteSparse;/opt/homebrew/lib/python3.12/site-packages/amici/lib64/cmake/SuiteSparse -DKLU_ROOT=/opt/homebrew/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/opt/homebrew/opt/python@3.12/bin/python3.12

==> Building:
$ cmak

In [6]:
loading = True

if loading:
    # load result history from file
    result = pypesto.store.read_result('optimization_history/n'+ str(n_runs) +'_mean.hdf5')

else:
    # optimize the model
    result = minimize(
        problem=problem,
        optimizer=FidesOptimizer(verbose=False, options={'maxiter': 5000}),
        n_starts=n_runs,
        engine=MultiProcessEngine(),
        # startpoint_method=uniform,
        history_options = pypesto.HistoryOptions(trace_record=True, storage_file='optimization_history/n'+ str(n_runs) +'_mean.hdf5'),
        filename='optimization_history/n'+ str(n_runs) +'_mean.hdf5',
    )

# print result summary
print(result.summary())

Loading the profiling result failed. It is highly likely that no profiling result exists within optimization_history/n5000_mean.hdf5.
Loading the sampling result failed. It is highly likely that no sampling result exists within optimization_history/n5000_mean.hdf5.


## Optimization Result 

* number of starts: 5000 
* best value: 1545.8758575253573, id=26
* worst value: inf, id=998
* number of non-finite values: 4238

* execution time summary:
	* Mean execution time: 0.315s
	* Maximum execution time: 29.376s,	id=3444
	* Minimum execution time: 0.008s,	id=3130
* summary of optimizer messages:

  |   Count | Message                                                     |
  |--------:|:------------------------------------------------------------|
  |    4238 | Encountered non-finite function inf value at initial point. |
  |     591 | Converged according to fval difference                      |
  |      97 | Trust Region Radius too small to proceed                    |
  |      74 | Converged according to gradient norm                        |

* best value found (approximately) 77 time(s)
* number of plateaus found: 29

A summary of the best run:

### Optimizer Result

* optimizer used: <FidesOptimizer  hessian_update=default verbose=False options={'

In [7]:
parameters_from_result = dict(zip(problem.x_names, result.optimize_result.list[0]['x']))

In [8]:
# Scale all parameters and put them into a dictionary
scaled_parameters = {key: 10**value for key, value in parameters_from_result.items()}

# Print the scaled parameters
print("Scaled parameters:")
for key, value in scaled_parameters.items():
    print(f"{key}: {value}")

Scaled parameters:
rho: 0.5778870607238772
kappa: 0.0012940690109779047
psi: 0.0001000000000000406
phi: 3.654793493459316
beta: 0.05293906993866318
alpha: 1.0894191289763626
delta: 9496.454285553269


# Obtain data and visualize the fitting result

In [9]:
return_dict = problem.objective(result.optimize_result.list[0].x, return_dict=True)
rdatas = return_dict['rdatas']
edatas = problem.objective.edatas
x_axis = [edata.id for edata in edatas]
simulation = [rdata.y.reshape(5, -1)[:,0] for rdata in rdatas]
data = [np.array(edata.getObservedData()).reshape(5, -1) for edata in edatas]

In [10]:
# Calculate residue errors for PBS and vvDD conditions
residue_errors_pbs_mean = [data[0][:, i] - simulation[0] for i in range(10)]
residue_errors_vvDD_mean = [data[1][:, i] - simulation[1] for i in range(10)]

In [11]:

petab_yaml = 'petab_files/individual_start.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

np.random.seed(500)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"INDIVIDUAL_START_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")
loading = True

if loading:
    # load result history from file
    result = pypesto.store.read_result('optimization_history/n'+ str(n_runs) +'.hdf5')

else:
    # optimize the model
    result = minimize(
        problem=problem,
        optimizer=FidesOptimizer(verbose=False, options={'maxiter': 5000}),
        n_starts=n_runs,
        engine=MultiProcessEngine(),
        # startpoint_method=uniform,
        history_options = pypesto.HistoryOptions(trace_record=True, storage_file='optimization_history/n'+ str(n_runs) +'.hdf5'),
        filename='optimization_history/n'+ str(n_runs) +'.hdf5',
    )

# print result summary
print(result.summary())
parameters_from_result = dict(zip(problem.x_names, result.optimize_result.list[0]['x']))
# Scale all parameters and put them into a dictionary
scaled_parameters = {key: 10**value for key, value in parameters_from_result.items()}

# Print the scaled parameters
print("Scaled parameters:")
for key, value in scaled_parameters.items():
    print(f"{key}: {value}")
    
# Obtain data and visualize the fitting result
return_dict = problem.objective(result.optimize_result.list[0].x, return_dict=True)
rdatas = return_dict['rdatas']
edatas = problem.objective.edatas
x_axis = [edata.id for edata in edatas]
simulation = [rdata.y.reshape(5, -1)[:,0] for rdata in rdatas]
data = [np.array(edata.getObservedData()) for edata in edatas]

Compiling amici model to folder /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/INDIVIDUAL_START_Model.
2024-10-22 18:20:06.261 - amici.petab.sbml_import - INFO - Importing model ...
2024-10-22 18:20:06.261 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-10-22 18:20:06.275 - amici.petab.sbml_import - INFO - Model name is 'INDIVIDUAL_START_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/INDIVIDUAL_START_Model'.
2024-10-22 18:20:06.276 - amici.petab.sbml_import - INFO - Species: 0
2024-10-22 18:20:06.277 - amici.petab.sbml_import - INFO - Global parameters: 18
2024-10-22 18:20:06.278 - amici.petab.sbml_import - INFO - Reactions: 0
2024-10-22 18:20:06.283 - amici.sbml_import - DEBUG - Finished validating SBML                    ++ (4.26E-04s)
2024-10-22 18:20:06.286 - amici.sbml_import - DEBUG - Finished converting SBML local parameters   ++ (8.92E-06s)
2024-10-22 18:20:06.286 - ami

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/INDIVIDUAL_START_Model -B /Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/INDIVIDUAL_START_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/opt/homebrew/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/models/7_individual_start/amici_models/0.26.1/INDIVIDUAL_START_Model/INDIVIDUAL_START_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_MODULE_PATH=/opt/homebrew/lib/python3.12/site-packages/amici/lib/cmake/SuiteSparse;/opt/homebrew/lib/python3.12/site-packages/amici/lib64/cmake/SuiteSparse -DKLU_ROOT=/opt/homebrew/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/opt/homebrew/opt/python@3.12/bin/python3.12

==> Building:
$ cmake --build /Users/yuhongliu/D

Loading the profiling result failed. It is highly likely that no profiling result exists within optimization_history/n5000.hdf5.
Loading the sampling result failed. It is highly likely that no sampling result exists within optimization_history/n5000.hdf5.


## Optimization Result 

* number of starts: 5000 
* best value: 1437.4088566250412, id=3690
* worst value: inf, id=998
* number of non-finite values: 4476

* execution time summary:
	* Mean execution time: 1.664s
	* Maximum execution time: 565.341s,	id=3099
	* Minimum execution time: 0.015s,	id=1974
* summary of optimizer messages:

  |   Count | Message                                                     |
  |--------:|:------------------------------------------------------------|
  |    4476 | Encountered non-finite function inf value at initial point. |
  |     401 | Converged according to fval difference                      |
  |      67 | Trust Region Radius too small to proceed                    |
  |      56 | Converged according to gradient norm                        |

* best value found (approximately) 53 time(s)
* number of plateaus found: 29

A summary of the best run:

### Optimizer Result

* optimizer used: <FidesOptimizer  hessian_update=default verbose=False options

In [ ]:
fig, axes = plt.subplots(10, 1, figsize=(10, 35), dpi=dpi)

# Plot for PBS
for i in range(10):
    ax = axes[i]
    ax.plot(data[i] - simulation[i], 'o', label='Invididual Start Model Residue' if i == 0 else '')
    ax.plot(residue_errors_pbs_mean[i], 'o', label='Mean Model Residue' if i == 0 else '')
    ax.set_title(f'Ctrl Individual {i+1}')
    ax.legend()

# Adjust the layout and remove empty legends
for ax in axes.flatten():
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 3))
    else:
        ax.legend().set_visible(False)

    ax.axhline(0, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.savefig(folder_path + 'residue_error_individual_vs_mean_ctrl.pdf', bbox_inches = 'tight', dpi=dpi)
plt.show()

In [ ]:
fig, axes = plt.subplots(10, 1, figsize=(10, 35), dpi=dpi)

# Plot for vvDD
for i in range(10, 20):
    ax = axes[i-10]
    ax.plot(data[i] - simulation[i], 'o', label='Invididual Start Model Residue' if i == 10 else '')
    ax.plot(residue_errors_vvDD_mean[i-10], 'o', label='Mean Model Residue' if i == 10 else '')
    ax.set_title(f'vvDD Individual {i-9}')
    ax.legend()

# Adjust the layout and remove empty legends
for ax in axes.flatten():
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 3))
    else:
        ax.legend().set_visible(False)

    ax.axhline(0, color='gray', linestyle='--', linewidth=1)

plt.tight_layout()
plt.savefig(folder_path + 'residue_error_individual_vs_mean_vvDD.pdf', bbox_inches = 'tight', dpi=dpi)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), dpi=dpi)

# Combine all residues for PBS
for i in range(20):
    ax.plot(data[i] - simulation[i], 'o', color='blue', label='Individual Start Model' if i == 0 else '')
    if i < 10:
        ax.plot(residue_errors_pbs_mean[i], 'o', color='orange', label='Mean Model' if i == 0 else '')
    else:
        ax.plot(residue_errors_vvDD_mean[i-10], 'o', color='orange')

# Adjust the layout and remove empty legends
handles, labels = ax.get_legend_handles_labels()
if handles:
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=2)
else:
    ax.legend().set_visible(False)

ax.axhline(0, color='gray', linestyle='--', linewidth=1)
ax.set_title('for both PBS and vvDD Conditions')
ax.set_xlabel('Time')
ax.set_ylabel('Residue Error')

ax.set_xticks(np.arange(0, 5, 1))
ax.set_xticklabels(np.arange(3, 8, 1))
plt.tight_layout()
plt.savefig(folder_path + 'combined_residue_error_individual_vs_mean.pdf', bbox_inches='tight', dpi=dpi)
plt.show()

In [23]:
mean_residual = (np.array(data) - np.array(simulation)).mean()
std_residual = (np.array(data) - np.array(simulation)).std()

print(f"Mean Residual: {mean_residual:.2f}")
print(f"Standard Deviation of Residuals: {std_residual:.2f}")

Mean Residual: -7932.62
Standard Deviation of Residuals: 422935.16


In [24]:
combined_residue_errors = np.concatenate((np.array(residue_errors_vvDD_mean), np.array(residue_errors_pbs_mean)), axis=0)
mean_combined_residue_error = combined_residue_errors.mean()
std_combined_residue_error = combined_residue_errors.std()

print(f"Mean Combined Residue Error: {mean_combined_residue_error:.2f}")
print(f"Standard Deviation of Combined Residue Errors: {std_combined_residue_error:.2f}")

Mean Combined Residue Error: 632.34
Standard Deviation of Combined Residue Errors: 1251458.11


In [27]:
from scipy.stats import f_oneway

# Calculate the residuals for data - simulation
residuals = [d - s for d, s in zip(data, simulation)]

# Flatten the lists for the F-test
residuals_flat = np.concatenate(residuals)
combined_residue_errors_flat = combined_residue_errors.flatten()

# Perform the F-test
f_statistic, p_value = f_oneway(residuals_flat, combined_residue_errors_flat)

print(f"F-statistic: {f_statistic:.2f}")
print(f"P-value: {p_value:.2e}")

# Check if the result is significant
alpha = 0.05
if p_value < alpha:
    print("The result is significant.")
else:
    print("The result is not significant.")

F-statistic: 0.00
P-value: 9.49e-01
The result is not significant.
